In [ ]:
import pandas as pd
from IPython.display import display, HTML
from jinja2 import Environment, FileSystemLoader
import os
import pathlib

In [ ]:
__file__ = 'src/qibocal/web/delete_me.ipynb'
WEB_DIR = pathlib.Path(__file__).parent
STYLES = WEB_DIR / "static" / "styles.css"
TEMPLATES = WEB_DIR / "templates"

In [ ]:
mydf = pd.DataFrame([{'a':1, 'b':2, 'c':3}])#, index=['hello'])

In [ ]:
mydf.to_html()#table_id=12, classes=['hello'], notebook=True)

In [ ]:
with open(STYLES) as file:
        css_styles = f"<style>\n{file.read()}\n</style>"

In [ ]:
mydf.style.css['row_heading'] = 'hi!'

In [ ]:
mydf = mydf.style.set_table_styles(
    [{'selector': 'thead th',
    'props': [('background-color', '#f0e6ff')]},
    # {'selector': 'thead th:first-child',
    # 'props': [('display','none')]},
    # {'selector': 'tbody th:first-child',
    # 'props': [('display','none')]}
    ]
)
# background-color: #f0e6ff;
#   padding: 8px;
#   text-align: center;
env = Environment(loader=FileSystemLoader(TEMPLATES))
template = env.get_template("test.html")
html = template.render(
    is_static=True,
    css_styles=css_styles,
    reports = mydf,
    )

In [ ]:
with open(os.path.join('delete_me/', "index.html"), "w") as file:
        file.write(html)

In [ ]:
HTML(html)

In [ ]:
import pandas as pd
import numpy as np

# Set up a DataFrame
np.random.seed(24)
df = pd.DataFrame({'A': np.linspace(1, 10, 10)})
df = pd.concat([df, pd.DataFrame(np.random.randn(10, 4), columns=list('BCDE'))],
               axis=1)
df.iloc[0, 2] = np.nan

df_html_output = df.style.set_table_styles(
    [{'selector': 'thead th',
    'props': [('background-color', '#f0e6ff'),
              ( 'padding', '8px'),
              ('text-align', 'center')]},
    # {'selector': 'thead th:first-child',
    # 'props': [('display','none')]},
    # {'selector': 'tbody th:first-child',
    # 'props': [('display','none')]}
    ]
).to_html()

HTML(df_html_output)
# html.append(df_html_output)
# body = '\r\n\n<br>'.join('%s'%item for item in html)
# msg.attach(MIMEText(body, 'html'))

In [172]:
from qibo.models import Circuit
from qibo import gates
import numpy as np

In [ ]:
c = Circuit(3)
c.add([gates.X(0), gates.Y(1), gates.CZ(0,2), gates.H(1), gates.M(0), gates.M(1)])

In [ ]:
nqubits = 3
qubits = [0,2]
layers = 4

gate_set = [gates.X, gates.Y, gates.H, gates.Unitary] #qasm

moments_int = np.random.randint(0, len(gate_set),size=(layers, len(qubits)))


init_kwargs_list = np.tile(np.arange(0, len(qubits)), layers).reshape((*moments_int.shape,1)).tolist()
# ? inverse ? what is faster? using matrices of gates or a analytic function?

collection_array = np.array([])
for l in init_kwargs_list:
    init_kwargs = np.empty(len(l), dtype=object)
    init_kwargs[:] = tuple(l)
    collection_array = np.append(collection_array, init_kwargs)
collection_array =  collection_array.reshape(layers, len(qubits))

In [ ]:
print(moments_int == 3)
for init_list in collection_array[moments_int == 3]:
    init_list.insert(0, np.random.uniform(0,1,(2,2)))

myc = Circuit(len(qubits))
for moment, moment_init in zip(moments_int, collection_array):
    for gate_index, init in zip(moment, moment_init):
        print(init)
        myc.add(gate_set[gate_index](*init))


In [ ]:
class sCircuit(Circuit):
    def __init__(self, nqubits, accelerators=None, density_matrix=False):
        super().__init__(nqubits, accelerators, density_matrix)

    def build(self, gate_set, moment_gates_indices, init_args=None):
        if init_args is None:
            init_args = np.tile(
                np.arange(0, len(qubits)), layers
            ).reshape((*moment_gates_indices.shape,1))

        for moment, moment_inits in zip(moment_gates_indices, init_args):
            for gate_index, init in zip(moment, moment_inits):
                self.add(gate_set[gate_index](*init))
    
    def compress(self):
        # ...
        # return gate_set, moment_gates_indices, init_args
        pass

    def save(self):
        # gate_set, moment_gates_indices, init_args =  self.compress()
        pass

    @classmethod
    def load(cls, path):
        # unpack file to gate_set, moment_gates_indices, init_args
        # obj = cls(len(moment_gates_inidices[0]))
        # obj.build(gate_set, moment_gates_indices, init_args)
        # return obj
        pass

In [174]:
from qibocal.auto.operation import Results
from dataclasses import dataclass, field
import numpy as np
from typing import List, Union
from qibocal.calibrations.niGSC.basics.fitting import fit_exp1B_func, exp1B_func
import plotly.graph_objects as go
from typing import Tuple, List


real_numeric = Union[int, float, np.number]
numeric = Union[int, float, complex, np.number]


# @dataclass
# class RResults(Results):
#     def render_html(self):
#         return "<div>"+self.__repr__()+"</div>"


@dataclass
class GSCResult(Results):
    key: str 
    summary: Results
    experiment_data: Union[str, List]

    def _plot(self):
        return self.summary._plot()

    def get_figures(self):
        return self.summary.get_figures()

    def get_tables(self):
        return self.summary.get_tables()
    # Handle not always load experimental data and pickle somewhere else. 

# TODO: MAYBE one class more!
# @dataclass
# class SingleDecayModel:
#     A: numeric = field(default=0.)
#     p: numeric = field(default=0.)
#     B: numeric = field(default=0.)
#     Aerr: numeric = field(default=0.)
#     perr: numeric = field(default=0.)
#     Berr: numeric = field(default=0.) 
#     fit: callable = fit_exp1B_func


@dataclass
class ResultContainer(Results):
    results: List[Results]
    summary: Results
    def _plot(self):
        for result in self.results:
            if hasattr(result, '_plot'):
                result._plot()

    def get_figures(self):
        figures = []
        for result in self.results:
            if hasattr(result, 'figure'):
                figures.extend(result.get_figures())
        return figures

    def get_tables(self):
        tables = []
        for result in self.results:
            if hasattr(result, 'get_tables'):
                tables.extend(result.get_tables())
        return tables


@dataclass
class DecayResult(Results):
    """
        y[i] = (A +- Aerr) (p +- perr)^m[i] + (B +- Berr)
        # for later: y= sum_i A_i p_i^m (needs m integer) 
    """
    y: List[numeric]
    m: List[numeric] = field(default_factory=list)
#    model: SingleDecayModel
    A: numeric = field(default=0.)
    Aerr: numeric = field(default=0.)
    p: numeric = field(default=0.)
    perr: numeric = field(default=0.)
    B: numeric = field(default=0.)
    Berr: numeric = field(default=0.)
#    figure: go.Figure = field(default_factory=go.Figure) #TODO: import plotly figure and do it!
    name: str = ''
    hist: Tuple[List[numeric], List[numeric], List[numeric]] = field(default_factory = lambda: (list(), list(), list()))
    fig: go.Figure = field(default_factory=go.Figure)

    def __post_init__(self):
        if len(self.m) == 0:
            self.m = list(range(len(self.y)))
        if len(self.y) != len(self.m):
            raise ValueError('Lenght of y and m must agree. len(m)={} != len(y)={}'.format(len(self.m), len(self.y)))

    def __str__(self):
        return 'DecayResult: {}, y = A p^m + B'.format(self.name)

    def _fit(self):
        params, errs, _ = fit_exp1B_func(self.m, self.y, p0=(self.A, self.p, self.B))
        self.A, self.p, self.B = params
        self.Aerr, self.perr, self.Berr = errs

    # can be defined by quicker by dataclass
    def _plot(self):
        self.fig = plot_decay_result(self)
        return self.fig

    def __str__(self):
        return u"({:.3f}\u00B1{:.3f})({:.3f}\u00B1{:.3f})^m + ({:.3f}\u00B1{:.3f})".format(
                self.A, self.Aerr, self.p, self.perr, self.B, self.Berr)

    def get_tables(self):
        return ['{} | {} '.format(self.name, str(self))]

    def get_figures(self):
        return [self.fig]

@dataclass
class MultiExponentialSeriesResult(Results): pass


# Design pattern, define result transformation
# Make transformation a type? 

def plot_decay_result(result: DecayResult, fig:go.Figure = None) -> go.Figure:
    if fig is None:
        fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=result.m,
            y=result.y,
            line=dict(color="#aa6464"),
            mode="markers",
            name="average",
        )
    )
    m_fit = np.linspace(min(result.m), max(result.m), 100)
    y_fit = exp1B_func(m_fit, result.A, result.p, result.B)
    fig.add_trace(
        go.Scatter(
            x=m_fit,
            y=y_fit,
            name=str(result),
        line=go.scatter.Line(dash="dot")
        )
    )
    return fig

def plot_decay_result_scatter(result: DecayResult): pass